In [6]:
%pip install "plotly" "nbformat>=4.2.0" "numpy" "kaleido"


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
queries = ['Q1', 'Q2', 'Q3','Q4','Q5','Q6','Q7']
binary_approaches = ['OCBC','OCCG', 'DECLARE']
approaches = ['OCPQ','DuckDB','SQLite','Neo4j', *binary_approaches] # , 'SQL'

In [8]:
import json
import numpy as np
res = {}
for q in queries:
    res[q] = {}
    for a in approaches:
        try:
            with open(f"./{q}/{a.lower()}-res.json") as f:
                res[q][a] = json.load(f)
        except:
                res[q][a] = False

In [9]:
res['Q1']['DECLARE'] = True
res['Q2']['DECLARE'] = True
res['Q3']['DECLARE'] = False
res['Q4']['DECLARE'] = False
res['Q5']['DECLARE'] = False
res['Q6']['DECLARE'] = False
res['Q7']['DECLARE'] = False



res['Q1']['OCCG'] = False
res['Q2']['OCCG'] = True
res['Q3']['OCCG'] = True
res['Q4']['OCCG'] = False
res['Q5']['OCCG'] = False
res['Q6']['OCCG'] = False
res['Q7']['OCCG'] = False


res['Q1']['OCBC'] = True
res['Q2']['OCBC'] = True
res['Q3']['OCBC'] = True
res['Q4']['OCBC'] = True
res['Q5']['OCBC'] = False # Not so sure, double check!
res['Q6']['OCBC'] = False
res['Q7']['OCBC'] = False

res['Q6']['SQLite'] = "⧗"

In [10]:
import plotly.graph_objects as go
import plotly.colors as colors_sq
import numpy as np
fig = go.Figure(data=go.Heatmap(
                    z=[[None if (a in binary_approaches or (res[q][a] == False) or (type(res[q][a]) == str)) else 1000 * np.mean(res[q][a]) for q in queries] for a in approaches],
                    x=queries,
                    y=[f"<b>{a}</b>" if a == "OCPQ" else a for a in approaches],
                    text=[
                            [
                                #str(res[q][a])
                                  "" if (a in binary_approaches or (res[q][a] == False)  or (type(res[q][a]) == str))  else
                            "" if np.isnan(np.mean(res[q][a])) else f"{np.round(1000 * np.mean(res[q][a]),0):.0f}ms"
                            for q in queries]
                          for a in approaches],
                    texttemplate="%{text}",
                    textfont={"size":20},
                    colorscale=colors_sq.sequential.YlGn_r,
                    # colorscale=colors_sq.sequential.speed_r,
                    # colorscale=colors_sq.sequential.gray,
                    # colorscale=colors_sq.sequential.Burgyl,
                    colorbar=dict(title=dict(text="Duration (ms)",side="bottom", font=dict(size = 16))),
                    ))
fig.update_yaxes(ticksuffix = " ", title = "Approach", showgrid=True, tickson='boundaries')
fig.update_xaxes(title = "Query", showgrid=True,tickson='boundaries')
fig.update_layout(font=dict(
        size=24,
    ),
    autosize=False,
    width=1000,
    height=500,
    template="plotly_white",
    title=""
    # color = "Duration",
    )
for q_i in range(len(queries)):
    for a_i in range(len(approaches)):
        q = queries[q_i]
        a = approaches[a_i]
        if a in binary_approaches or res[q][a] == False or type(res[q][a]) == str:
            fig.add_annotation(
                x=q_i,y=a_i,text="✔" if res[q][a] == True else "✗" if res[q][a] == False else res[q][a],
                showarrow=False,
                font=dict(size=32, color="green" if res[q][a] == True else "lightgray" if res[q][a] == False else "gray"))
            # or lightsalmon
fig.show()
fig.write_image("heatmap.svg")
fig.write_image("heatmap.pdf")
fig.write_image("heatmap.png")